This Notebook is for plotting parts of the PAP algorithm, to compare with the original paper by Pouchou and Pichoir (1991).

In [ ]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go

from PAP_functions.PAP_helper_functions import theoretical_energy, get_C_A_Z_arrays
from PAP_functions.PAP_area_F import set_m_small, ionization_cross_section_Q, mean_atomic_mass_M, mean_ionzation_potential_J, energy_dependent_terms_f_of_v, energy_loss_dE_drhos, whole_dE_drhos, deceleration_factor_one_over_S, backscattering_factor_R, mean_atomic_number_Zb, mean_backscattering_coefficient_eta

In [ ]:
hs.material.elements.Ga.Atomic_properties.Xray_lines.Ka['energy (keV)']

In [ ]:
## Looking at the values

elements = ['Ga', 'Sb']
line = 'Ga_La'
e0 = 15  # keV
concentrations = [0.5, 0.5]  # at%

e_c = theoretical_energy(line=line)
u = e_c / e0
m_small = set_m_small(line=line)

array_C, array_A, array_Z = get_C_A_Z_arrays(elements=elements, 
                                         concentrations=concentrations, concentration_type='at')
q = ionization_cross_section_Q(e0=e0, line=line)
m = mean_atomic_mass_M(array_C=array_C, array_Z=array_Z, array_A=array_A)
j = mean_ionzation_potential_J(array_C=array_C, array_Z=array_Z, array_A=array_A)
f_of_v = energy_dependent_terms_f_of_v(e=e0, j=j)
dE_drhos = energy_loss_dE_drhos(m=m, j=j, f_of_v=f_of_v)
dE_drhos2 = whole_dE_drhos(array_C=array_C, array_Z=array_Z, array_A=array_A, e0=e0)

print(f'C wt%: {array_C}')
print(f'A Da: {array_A}')
print(f'Z: {array_Z}')
print(f'Q: {q:.3f}')
print(f'M: {m:.3f}')
print(f'J: {j:.3f}')
print(f'1/f(V): {1/f_of_v:.3f}')

In [ ]:
def plot_dE_drhos(*, fig, elements, at_list, energies, name):
    array_C, array_A, array_Z = get_C_A_Z_arrays(elements=elements, 
                                         concentrations=at_list, concentration_type='at')
    m = mean_atomic_mass_M(array_C=array_C, array_Z=array_Z, array_A=array_A)
    j = mean_ionzation_potential_J(array_C=array_C, array_Z=array_Z, array_A=array_A)
    f_of_v_list = [energy_dependent_terms_f_of_v(e=e, j=j) for e in energies]
    de_drhos_list = np.array([energy_loss_dE_drhos(m=m, j=j, f_of_v=f) for f in f_of_v_list])
    fig.add_trace(go.Scatter(x=energies, y=de_drhos_list, name=name))


In [ ]:
energies = np.linspace(0.01, 50, 10000)
# f_of_v_list = [energy_dependent_terms_f_of_v(e=e, j=j) for e in energies]
# de_drhos_list = np.array([energy_loss_dE_drhos(m=m, j=j, f_of_v=f) for f in f_of_v_list])

fig = go.Figure()
plot_dE_drhos(fig=fig, elements=['Ga', 'Sb'], at_list=[0.5, 0.5], energies=energies, name='GaSb')
plot_dE_drhos(fig=fig, elements=['Ga', 'As'], at_list=[0.5, 0.5], energies=energies, name='GaAs')
plot_dE_drhos(fig=fig, elements=['Cu'], at_list=[1.], energies=energies, name='Cu (PAP Fig. 4)')

fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"),
                  xaxis_title="Energy [keV]", xaxis_type="log",
                  yaxis_title="dE/d\u03C1s [keV cm<sup>2</sup>/g]",
                  margin=dict(l=5, r=5, b=5, t=5), width=800, height=400,
                  legend=dict(x=0.995, y=0.99, xanchor='right',),
                  ) 

In [ ]:
# fig.write_image('../mastersthesis/figures/PAP_energy_loss_dE_drhos.pdf')
# # fig.write_image('../mastersthesis/figures/PAP_deceleration_of_electrons.pdf')  # old name


In [ ]:
# Making a Q(U) plot

e0_max = 30
def add_line_q_of_u(fig, line, name):
    e_c = theoretical_energy(line=line)
    e0_list = np.arange(e_c, e0_max, 0.01)
    u_list = e0_list / e_c
    q_list = [ionization_cross_section_Q(e0=e0, line=line) for e0 in e0_list]
    print('max q =', round(max(q_list),3), ' at U0 =' , round(u_list[q_list.index(max(q_list))],3), 
    ' at E =', round(e0_list[q_list.index(max(q_list))], 3), 'keV')
    fig.add_trace(go.Scatter(x=u_list, y=q_list, name=f'{name}, E<sub>C</sub>{e_c:.2f} keV'))

fig = go.Figure()
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"))
add_line_q_of_u(fig, 'Ga_La', 'Ga L\u03B1')
add_line_q_of_u(fig, 'As_La', 'As L\u03B1')
fig.update_layout( xaxis_title='U = E<sub>0</sub>/E<sub>c</sub>', yaxis_title='Q<sub> l</sub><sup>A</sup>(U)',
                  margin=dict(l=5, r=5, b=5, t=5), width=800, height=400,
                  legend=dict(x=0.995, y=0.99, xanchor='right',),
                  xaxis_range=[-0.1, 25])

fig.add_annotation(x=3.38, y=0.372, text="Q<sub> max</sub> at  (3.38, 0.37)<br> i.e. E<sub>0</sub> = 3.7 keV", ax=275, ay=0)
fig.add_annotation(x=3.38, y=0.273, text="Q<sub> max</sub> at  (3.38, 0.27)<br> i.e. E<sub>0</sub> = 4.3 keV", ax=275, ay=130)

# fig.write_image('../mastersthesis/figures/PAP_ionization_cross_section.pdf')

fig 

In [ ]:
# calculate R for all Z, with varying U0

array_U = [1.1, 1.3, 1.5, 2, 3, 5, 20]
array_all_Z = np.arange(1,90)
array_all_R = []
for z in array_all_Z:
    array_C = [1]
    array_Z = [z]
    array_R = []
    for u in array_U:
        array_R.append(backscattering_factor_R(u0=u, array_C=array_C, array_Z=array_Z))
    array_all_R.append(array_R)    
array_all_R = np.array(array_all_R)

fig = go.Figure()
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"),
                xaxis_title="Z", yaxis_title="Backscattering factor R", legend_title='Overvoltage',
                margin=dict(l=5, r=5, b=5, t=5), width=800, height=400,)

for i, u0 in enumerate(array_U):
    fig.add_trace(go.Scatter(x=array_all_Z, y=array_all_R[:,i], name=f"{u0}"))


# fig.write_image('../mastersthesis/figures/PAP_backscattering_factor.pdf')

fig

In [ ]:
# Surface ionization

# calculation of phi_zero
def phi_zero__surface_ionization_2(*, e0: float, e_c: float, array_C: np.array, array_Z: np.array):
    zb = mean_atomic_number_Zb(array_C=array_C, array_Z=array_Z)
    eta = mean_backscattering_coefficient_eta(zb=zb)
    u = e0 / e_c
    return 1 + 3.3 * (1 - 1 / (u**(2 - 2.3 * eta))) * eta**1.2

def plot_phi_zero():
    # remaking plot 24 from the paper
    e0_list = [1.1, 1.3, 1.5, 2, 3, 5, 20, 100]
    e_c = 1 # one to get the overvoltages in the paper
    all_z = np.arange(1, 90, 1)
    array_C = [1]
    fig = go.Figure()
    for e0 in e0_list:
        y = [phi_zero__surface_ionization_2(e0=e0, e_c=e_c, array_C=array_C, array_Z=[z]) for z in all_z]
        fig.add_trace(go.Scatter(x=all_z, y=y, name=f'{e0}'))
    return fig

fig = plot_phi_zero()
fig.update_layout(xaxis_title='Z', yaxis_title='Surface ionization potential, \u03C6(0)',
                    legend=dict(traceorder='reversed', title='Overvoltage'))
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"),
                    margin=dict(l=5, r=5, b=5, t=5), width=800, height=400)

# fig.write_image('../mastersthesis/figures/PAP_surface_ionization_potential.pdf')
fig